In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [44]:
#data cleaning
df = pd.read_json('../data/fizzy.json', lines=True)
df['actualCostEth'] = pd.to_numeric('0.' + df.actualCost.str.split(' ').str[0])
df['actualCostUsd'] = pd.to_numeric(df.actualCost.str.split(' ').str[2].str.slice(2,6,1))
del df['actualCost']
df.blockHeight = pd.to_numeric(df.blockHeight)
df.gasLimit = pd.to_numeric(df.gasLimit)
df.gasPrice = pd.to_numeric('0.' + df.gasPrice.str.split(' ').str[0])
df.gasUsed = pd.to_numeric(df.gasUsed.str.split(' ').str[0])
df.value = pd.to_numeric(df.value.str.slice(0,1,1))
del df['txReceiptStatus']
df.head()

,blockHeight,fromAddress,gasLimit,gasPrice,gasUsed,inputData,nonce,timeStamp,txHash,value,actualCostEth,actualCostUsd
0,6861211,0x50e00de2c5cc4e456cf234fcb1a0efa367ed016e,132061,9.000000e-09,120055,"Function: addNewInsurance(bytes32 flightId, ui...",19564,1 day 1 hr ago (Dec-10-2018 01:58:45 PM +UTC),0xe053c29b2e6c7a72837088107ca5ccc76768085bf713...,0,0.001080,0.09
1,6861212,0x50e00de2c5cc4e456cf234fcb1a0efa367ed016e,132061,9.000000e-09,120055,"Function: addNewInsurance(bytes32 flightId, ui...",19565,1 day 1 hr ago (Dec-10-2018 01:59:04 PM +UTC),0x62a5f72510f12583d13ac5cfd9cd19991f3521021c2e...,0,0.001080,0.09
2,6836388,0x50e00de2c5cc4e456cf234fcb1a0efa367ed016e,115561,9.000000e-09,105055,"Function: addNewInsurance(bytes32 flightId, ui...",19548,5 days 3 hrs ago (Dec-06-2018 11:41:57 AM +UTC),0x382809be94f8b8226bd787768dcb7f1056181bf8c81a...,0,0.000945,0.08
3,6839275,0x50e00de2c5cc4e456cf234fcb1a0efa367ed016e,38427,9.000000e-09,34934,"Function: updateFlightStatus(bytes32 flightId,...",19552,4 days 16 hrs ago (Dec-06-2018 11:06:44 PM +UTC),0xdbac6eb83606989389b23dfe53405ff8152e2e8eb2f6...,0,0.000314,0.03
4,6840920,0x50e00de2c5cc4e456cf234fcb1a0efa367ed016e,38639,9.000000e-09,35126,"Function: updateFlightStatus(bytes32 flightId,...",19553,4 days 9 hrs ago (Dec-07-2018 05:46:52 AM +UTC),0x9c447fb74e779051e9f47576640cc95a0734ee6e7a20...,0,0.000316,0.03
